# LangGraph Analyst Workflow

In [1]:
import bql
import boto3
import botocore
import random
import json
import importlib
from utils.s3_helper import S3Helper

from langchain.prompts import PromptTemplate
from langchain_core.runnables import RunnableSequence
from langchain_core.rate_limiters import InMemoryRateLimiter

from langchain_aws import ChatBedrock
from pydantic import BaseModel, Field
from IPython.display import Markdown, display
import company_data
from datetime import datetime
from dateutil.relativedelta import relativedelta

import prompts
import pandas as pd

import concurrent.futures

from langgraph.graph import START, END, StateGraph
from langgraph.graph.message import add_messages

from typing import Dict, TypedDict, Optional

In [2]:
bq = bql.Service()

In [3]:
# Configure Bedrock
# Bedrock client initialise
config = botocore.config.Config(read_timeout=1000)
boto3_bedrock = boto3.client('bedrock-runtime',config=config)

with open('Data/prompts.json', 'rb') as f:
    prompt_set = json.load(f)

### Configure the Models

In [97]:
model_claude_id = 'us.anthropic.claude-3-7-sonnet-20250219-v1:0'
model_claude_small = "us.anthropic.claude-3-5-haiku-20241022-v1:0"
model_llama_id = 'us.meta.llama3-1-70b-instruct-v1:0'
model_llama_small_id = "us.meta.llama3-1-8b-instruct-v1:0"

# set up the LLM in Bedrock
rate_limiter = InMemoryRateLimiter(
    requests_per_second=50,
    check_every_n_seconds=1,
    max_bucket_size=10,
)

llm_thinker = ChatBedrock(
    client = boto3_bedrock,
    model_id = model_claude_id,
    temperature = 0.01,
    max_tokens=4000,
    rate_limiter = rate_limiter
)

llm_small = ChatBedrock(
    client = boto3_bedrock,
    model_id = model_llama_small_id,
    temperature = 0.01,
    max_tokens = 4000,
    rate_limiter = rate_limiter
)

llm_debate = ChatBedrock(
    client = boto3_bedrock,
    model_id = model_claude_small,
    temperature = 0.01,
    max_tokens = 4000,
    rate_limiter = rate_limiter
)

## Test Company Prompt

## Prompts

In [42]:
statement_analysis_system_prompt = prompts.SYSTEM_PROMPTS['COT_CLAUDE_TEST']['prompt']
statement_analysis_template = PromptTemplate.from_template(statement_analysis_system_prompt)

clean_headlines_system_prompt = """You are an assistant to a financial analyst analyzing {security} You must remove any reference to {security} and their products from the following list of headlines and replace them with the term 'blah'. Replace the names of any people such as ceo in the article with the term 'whah' do not refer to {security} at all in your answer:{headlines}"""
clean_headlines_template = PromptTemplate.from_template(clean_headlines_system_prompt)

company_news_system_prompt = """You are a financial analyst and are reviewing news for company called blah over the last three months. Blah is in the {sector} sector. Start by listing the revenue drivers for the sector. Then look through the below headlines and determine if blah will see an increase or decrease in their earnings over the next quarter. Think through your response. {headlines}"""
company_news_template = PromptTemplate.from_template(company_news_system_prompt)

senior_analysis_prompt = """You are a senior financial analyst and review your teams work. You are looking at a financial summary and news for 'blah'. Using the summaries only, critique the report and construct an alternative narrative. If the narrative is in agreement with the two reports, make clear your belief in the direction of earning. If in disagreement, state why you disagree. Think through your response. {financial_summary} \n {news_summary}"""
senior_analyst_template = PromptTemplate.from_template(senior_analysis_prompt)

## LangGraph Implementation

In [89]:
def analyst_llm(llm, prompt):
    return llm.invoke(prompt).content

In [107]:
class CompanyData(TypedDict):
    name: str
    figi_name: str
    sector: str
    sec_fs: str
    headlines: list[str]
    stock_prices: str

class GraphState(TypedDict):
    company_details: Optional[dict[str,str]]
    initial_analysis: Optional[str]
    cleaned_headlines: Optional[list]
    news_report: Optional[str]
    senior_report: Optional[str]

analyst_workflow = StateGraph(GraphState)

In [108]:
def financial_statement_analysis(state):
    # Create the prompt to feed into the model
    company_details = state.get('company_details')
    sec_fs = company_details['sec_fs']
    prompt_in = statement_analysis_template.format(financials=sec_fs)
    financial_analysis = analyst_llm(llm_thinker, prompt_in)
    return {'initial_analysis': financial_analysis}

def clean_headlines(state):
    company_details = state.get('company_details')
    unclean_headlines = company_details['headlines']
    # Create the prompt to feed into the model
    prompt_in = clean_headlines_template.format(headlines=unclean_headlines, security=name)
    clean_headlines = analyst_llm(llm_small, prompt_in)
    return {'cleaned_headlines': clean_headlines}

def news_summary(state):
    # Create the prompt to feed into the model
    company_details = state.get('company_details')
    clean_headlines = state.get('cleaned_headlines')
    prompt_in = company_news_template.format(headlines=clean_headlines[1:], sector=company_details['sector'])
    news_summarisation = analyst_llm(llm_thinker, prompt_in)
    return {'news_summary': news_summarisation}

def final_report(state):
    company_details = state.get('company_details')
    initial_analysis = state.get('initial_analysis')
    news_summary = state.get('news_summary')
    prompt_in = senior_analyst_template.format(financial_summary=initial_analysis, news_summary=news_summary)
    final_report_output = analyst_llm(llm_thinker, prompt_in)
    return {'senior_report': final_report_output}

In [109]:
analyst_workflow.add_node('financial_statement_analysis', financial_statement_analysis)
analyst_workflow.add_node('clean_headlines', clean_headlines)
analyst_workflow.add_node('news_summary', news_summary)
analyst_workflow.add_node('final_report', final_report)

In [110]:
analyst_workflow.set_entry_point('financial_statement_analysis')
analyst_workflow.add_edge('financial_statement_analysis', 'clean_headlines')
analyst_workflow.add_edge('clean_headlines', 'news_summary')
analyst_workflow.add_edge('news_summary', 'final_report')
analyst_workflow.add_edge('final_report', END)

In [111]:
app = analyst_workflow.compile()

In [112]:
app.get_graph()

Graph(nodes={'__start__': Node(id='__start__', name='__start__', data=<class 'langchain_core.utils.pydantic.LangGraphInput'>, metadata=None), 'financial_statement_analysis': Node(id='financial_statement_analysis', name='financial_statement_analysis', data=financial_statement_analysis(tags=None, recurse=True, explode_args=False, func_accepts_config=False, func_accepts={}), metadata=None), 'clean_headlines': Node(id='clean_headlines', name='clean_headlines', data=clean_headlines(tags=None, recurse=True, explode_args=False, func_accepts_config=False, func_accepts={}), metadata=None), 'news_summary': Node(id='news_summary', name='news_summary', data=news_summary(tags=None, recurse=True, explode_args=False, func_accepts_config=False, func_accepts={}), metadata=None), 'final_report': Node(id='final_report', name='final_report', data=final_report(tags=None, recurse=True, explode_args=False, func_accepts_config=False, func_accepts={}), metadata=None), '__end__': Node(id='__end__', name='__end_

## Test on an example firm

In [113]:
# test prompt
security = prompt_set[0]
sec_name = security['security']
sec_date = security['date']
sec_fs = security['prompt'][1]['content']
sec_prices = sec_fs[sec_fs.find('Historical Price:') + 17:]

In [83]:
# get the news headlines
s3_helper = S3Helper('tmp/fs')
s3_helper.get_file(filename='dow_headlines.parquet', local_filename='/tmp/dow_headlines.parquet')
news_headlines = pd.read_parquet('/tmp/dow_headlines.parquet')

In [84]:
# Convert sec_name to figi
def get_additional_data(sec_name: str) -> str:
    """Function to convert Bloomberg tickers in a dataframe to FIGIs for ESL"""
    univ      = sec_name
    field     = {'figi': bq.data.composite_id_bb_global(), 
                 'name': bq.data.name(), 
                 'sector': bq.data.bics_level_1_sector_name()}
    req = bql.Request(univ, field)
    data = bq.execute(req)
    return (data[0].df().loc[sec_name]['figi'], data[1].df().loc[sec_name]['name'], data[2].df().loc[sec_name]['sector'])

figi_name, name, sector = get_additional_data(sec_name)
    

In [85]:
def filter_by_company_date(dataset, security, max_date = None):
    
    if max_date != None:
        #claculate the minimum date to get a 3 month window
        min_date = datetime.strptime(max_date,"%Y-%m-%d") + relativedelta(months=-3)
        min_date = min_date.strftime("%Y-%m-%d")
        filtered_dataset = dataset[(dataset['TimeOfArrival'] < max_date) &  (dataset['TimeOfArrival'] >= min_date) & (dataset['Assigned_ID_BB_GLOBAL'] == security)]
    else:
        filtered_dataset = dataset[dataset['Assigned_ID_BB_GLOBAL'] == security]
    
    filtered_list = filtered_dataset['Headline'].to_list()

    if len(filtered_list) >= 50:
        return random.sample(filtered_list, 50)
    else:
        return filtered_list

In [86]:
headline_filter_test = filter_by_company_date(news_headlines, figi_name, sec_date)

In [87]:
company_details = CompanyData({'name':name,
                              'figi_name': figi_name,
                              'sector': sector,
                              'sec_fs': sec_fs,
                              'headlines': headline_filter_test,
                              'stock_prices': sec_prices})

In [114]:
%%time
conversation = app.invoke({'company_details': company_details})

In [115]:
conversation['senior_report']

'# Financial Analysis Review: Earnings Forecast Critique\n\n## Executive Summary\n\nAfter reviewing the financial analysis report for \'blah\', I find the overall conclusion of a 4-6% earnings decrease in the next period to be reasonable, though I believe the analysis could benefit from a more balanced interpretation of certain metrics. The report correctly identifies concerning profitability trends, but may underweight some positive efficiency indicators that could partially offset these negative factors.\n\n## Strengths of the Analysis\n\nThe report provides a comprehensive review of key financial metrics and correctly identifies several concerning trends:\n\n1. The consistent decline in profitability metrics (gross margin, operating margin, net margin) over multiple periods is indeed troubling\n2. The significant increase in debt ratios, particularly the jump in debt-to-equity from 0.44 to 1.73\n3. The declining interest coverage ratio (from 9.79 to 6.15) which signals increased fin

## Test with financial_agent class

In [116]:
import company_data as cd

In [117]:
importlib.reload(cd)

<module 'company_data' from '/project/company_data.py'>

In [120]:
sec_data = cd.SecurityData('tmp/fs', 'dow_quarterly_ltm.json')

Exception: Company data must be downloaded first before it is requested.